In [2]:
import numpy
from etrade.spot.trader import Station
from etrade.spot.market.recycle import BasicRecycle, PointwiseRecycle
from etrade.spot.forecast.market import DistributiveSeries, DistributiveMarket

from data_utils.stochastic_utils.vdistributions.parameter.continuous.basic import NormalDistribution, LogNormalDistribution, eps
from data_utils.stochastic_utils.vdistributions.nonparametric.continuous.histogram import HistogramDistribution
from data_utils.stochastic_utils.vdistributions.nonparametric.discrete.basic import DiscreteDistribution
from data_utils.stochastic_utils.vdistributions.tools.convert import resample_like_distribution, generate_correlated_sample_matrix

from matplotlib import pyplot as plt

from scipy.optimize import differential_evolution

from matplotlib import pyplot

In [6]:
s = Station("station", 50)
monte_carlo_num = 1000


In [4]:
def real_market_data(data_len = 4, num = 100):
    aq_mean = NormalDistribution(25, 10).rvf(data_len * num)
    aq_std = NormalDistribution(5, 1).rvf(data_len * num)

In [5]:
real_aq_data

array([[25,  2],
       [30,  3],
       [40,  4],
       [10,  2]])